In [ ]:
import pandas as pd

In [ ]:
schools = pd.read_json('https://stem-mentoring.firebaseio.com/school.json')
cities = pd.read_json('https://stem-mentoring.firebaseio.com/cities.json')
students = pd.read_json('https://stem-mentoring.firebaseio.com/students.json').T
confirm = pd.read_json('https://stem-mentoring.firebaseio.com/confirm.json').T

In [ ]:
students['school_id'] = students.school.map(lambda x:x['id'])
students['school_name'] = students.school.map(lambda x:x['label'])
students['city_id'] = students.school.map(lambda x:x['city'])

In [ ]:
students.columns

# The content below is real code that generates list of participants.

In [ ]:
students = pd.read_json('https://stem-mentoring.firebaseio.com/students.json').T
students['school_name'] = students.school.map(lambda x:x['value'])
register_list = students[['school_name', 'order', 'name', 'grade', 'phone', 'guardian_name', 'guardian_phone', 'interests', 'motivation', 'timestamp']].sort_values(['school_name', 'order'])
register_list.to_excel('명단.xlsx', header=['학교', '순번', '이름', '학년', '연락처', '보호자', '보호자 연락처', '관심 학과', '동기', '접수 시간'], index=False)

## 학교 목록 csv 파일로부터 json을 생성합니다. 매년 멘토링 전 한 번만 하세요!

In [ ]:
import csv
from collections import defaultdict
import json

with open('cities.json', 'r') as cityfile:
    cities = json.loads(cityfile.read())

code_ref = defaultdict(dict)

for city in cities:
    code_ref[city['province']][city['city']] = city['city_id']

keys = ('시도','행정구','학교명','도로명주소')

schools = []
name_collection = defaultdict(list)


with open('list.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    title = next(reader)
    PROV = title.index(keys[0])
    CITY = title.index(keys[1])
    NAME = title.index(keys[2])
    ADDR = title.index(keys[3])

    for row in reader:
        schools.append({
            'city_id': code_ref[row[PROV]][row[CITY]],
            'count': 0,
            'name': row[NAME]
        })
        name_collection[row[NAME]].append(len(schools)-1)

for k, v in name_collection.items():
    if len(v) > 1:
        print(k, '(%d)' % len(v), end=': ')
        for i in v:
            schools[i]['name'] += ' (%s)' % (cities[schools[i]['city_id']]['city'])
            print (schools[i]['name'], end=', ')
        print()

with open('schools.json', 'w') as schoolfile:
    json.dump(schools, schoolfile, ensure_ascii=False, indent=2)

In [ ]:
#recover city list

import csv
from collections import defaultdict
import json

keys = ('시도','행정구','학교명')


cities = []

with open('list.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    title = next(reader)
    PROV = title.index(keys[0])
    CITY = title.index(keys[1])
    for row in reader:
        district = '%s/%s' % (row[PROV], row[CITY])
        if district not in cities:
            cities.append(district)
for i in range(len(cities)):
    info = cities[i].split('/')
    cities[i] = {
        'city_id': i,
        'city': info[1],
        'province': info[0]
    }

with open('cities.json', 'w') as cityfile:
    json.dump(cities, cityfile, ensure_ascii=False, indent=2)
    